In [1]:
#we will run lda on the whole wiki sample and ck12

import gensim
import logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = unicode(line, encoding='utf-8', errors='replace')
                yield line.split()
                
                
path = '/Users/MK/GitHub/the_answer_is/data/temporary4'
sentences = MySentences(path)

dictionary = corpora.Dictionary(sentences)

In [2]:
#define stopwords 
stoplist = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours',
             u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', 
             u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', 
             u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', 
             u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', 
             u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', 
             u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', 
             u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', 
             u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', 
             u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', 
             u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', 
             u'should', u'now']

In [3]:


print 'prior to removing words that only appear once:'
print dictionary
#remove infrequent words in the dictionary
#remove stopwords in the dictionary
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.compactify() # remove gaps in id sequence after words that were removed
dictionary
print 'after removing: '
print dictionary

prior to removing words that only appear once:
Dictionary(394 unique tokens: [u'essay', u'influenza', u'founder', u'lack', u'abil']...)
after removing: 
Dictionary(101 unique tokens: [u'influenza', u'reservoir', u'less', u'endem', u'cold']...)


In [4]:
print(dictionary.token2id)

{u'influenza': 0, u'reservoir': 1, u'less': 2, u'endem': 3, u'cold': 4, u'either': 6, u'spread': 7, u'sourc': 8, u'march': 9, u'epidem': 10, u'period': 52, u'mechan': 12, u'gate': 13, u'increas': 14, u'affect': 15, u'requir': 16, u'introduct': 17, u'term': 18, u'success': 20, u'organ': 19, u'vector': 21, u'common': 22, u'refer': 23, u'mean': 24, u'respond': 25, u'vehicl': 26, u'epidemiolog': 27, u'eg': 28, u'rate': 29, u'respons': 30, u'guarante': 31, u'transmiss': 32, u'safeti': 33, u'abl': 34, u'may': 88, u'health': 36, u'condit': 37, u'pandem': 38, u'diseas': 39, u'persontoperson': 40, u'genet': 41, u'water': 42, u'host': 43, u'vertic': 44, u'popul': 45, u'materi': 70, u'contamin': 47, u'place': 48, u'mani': 49, u'chang': 50, u'undergo': 51, u'number': 11, u'one': 53, u'feet': 54, u'system': 55, u'next': 56, u'attack': 57, u'includ': 58, u'way': 59, u'time': 99, u'individu': 60, u'form': 61, u'infect': 62, u'suscept': 63, u'surveil': 64, u'case': 65, u'exposur': 66, u'novel': 67, u'

In [5]:
#if you do not care about the memory, just do
# corpus = [dictionary.doc2bow(word) for word in sentences]

class MyCorpus(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = unicode(line, encoding='utf-8', errors='replace')
                yield dictionary.doc2bow(line.lower().split())

path = '/Users/MK/GitHub/the_answer_is/data/temporary4'
corpus = MyCorpus(path)              

In [7]:
#model saving
corpora.MmCorpus.serialize('/Users/MK/GitHub/the_answer_is/models/lda/test_corpus_saved.mm', corpus)
#load_corpus = corpora.MmCorpus('/Users/MK/GitHub/the_answer_is/models/lda/wiki_ck_corpus_saved.mm')


In [6]:
print corpus

In [18]:
lda = gensim.models.LdaModel(corpus, id2word = dictionary, num_topics=3
                            )
#lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=100, update_every=1, chunksize=10000, passes=1)

In [19]:
lda.print_topics(3)

[(0,
  u'0.042*diseas + 0.038*outbreak + 0.034*emerg + 0.033*transmiss + 0.032*host + 0.029*exposur + 0.028*common + 0.024*sourc + 0.023*worker + 0.021*epidem'),
 (1,
  u'0.089*epidem + 0.039*diseas + 0.028*peopl + 0.025*may + 0.023*occur + 0.023*spread + 0.021*infect + 0.019*common + 0.017*health + 0.017*outbreak'),
 (2,
  u'0.108*transmiss + 0.055*agent + 0.032*vehicl + 0.027*develop + 0.027*contamin + 0.026*spread + 0.026*mechan + 0.026*biolog + 0.026*propag + 0.025*infect')]

In [11]:
#'courtesy is mapped to 3712 in the dictionary, 
# flickrbalancedcrafts is mapped to 1 in the dictionary. 
# if we have a document with 100 words of courtesy, and 1 word of flickrbalancedcrafts,
# the document is represented as below, and is close to the topic 18, which contains the word coutesy with weight 0.085 
doc_bow = [(3712, 100), (1, 1)]
lda[doc_bow]

[(102, 0.98044119860059198)]

In [12]:
import pandas as pd
import os
path = '/Users/MK/GitHub/the_answer_is/data'
os.chdir(path)
train = pd.read_table('training_set.tsv',sep = '\t')
train.head()

,id,question,correctAnswer,answerA,answerB,answerC,answerD
0,100001,"When athletes begin to exercise, their heart r...",C,at the tissue level,at the organ level,at the system level,at the cellular level
1,100002,Which example describes a learned behavior in ...,C,smelling the air for odors,barking when disturbed,sitting on command,digging in soil
2,100003,"When two nuclei are combined into one nucleus,...",D,conversion,reaction,fission,fusion
3,100004,Which is a distinction between an epidemic and...,B,the symptoms of the disease,the geographical area affected,the species of organisms infected,the season in which the disease spreads
4,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...


In [13]:
i = 2
q = train.ix[i][1].split()
a1 = train.ix[i][3].split()
a2 = train.ix[i][4].split()
a3 = train.ix[i][5].split()
a4 = train.ix[i][6].split()
    
print q,a1,a2,a3,a4

['When', 'two', 'nuclei', 'are', 'combined', 'into', 'one', 'nucleus,', 'there', 'is', 'a', 'slight', 'change', 'in', 'mass', 'and', 'the', 'release', 'of', 'a', 'large', 'amount', 'of', 'energy.', 'What', 'is', 'this', 'process', 'called?'] ['conversion'] ['reaction'] ['fission'] ['fusion']


In [14]:
#convert question into corpus and run lda. 
corpus = dictionary.doc2bow(q)
lda[corpus]

[(8, 0.083749999999999811),
 (57, 0.088109974441187372),
 (59, 0.084843451326104283),
 (86, 0.20425121521302006),
 (111, 0.083749999999999811),
 (115, 0.06111558918919191),
 (148, 0.095639923595379248),
 (176, 0.1260330324061251),
 (196, 0.092923480495657115)]

In [15]:
indexlist = [x[0] for x in lda[corpus]]
indexlist

[8, 57, 59, 86, 111, 115, 148, 176, 196]

In [16]:
for index in indexlist:
    print lda.print_topic(index)
    
#this is the words that are in the topics which are close to the quesion. 

0.298*mass + 0.122*data + 0.082*smaller + 0.081*center + 0.052*mean + 0.038*images + 0.036*entire + 0.024*taxonomy + 0.020*satellites + 0.019*machine
0.286*process + 0.170*produce + 0.067*ice + 0.039*may + 0.029*called + 0.022*rotational + 0.020*stronger + 0.018*efficient + 0.018*poor + 0.016*transported
0.231*changes + 0.071*rock + 0.070*away + 0.057*release + 0.050*eruptions + 0.039*growing + 0.033*reduction + 0.032*estimate + 0.032*sediment + 0.030*trait
0.125*rather + 0.062*whose + 0.060*consists + 0.052*idea + 0.049*better + 0.044*combined + 0.041*devices + 0.032*stages + 0.029*older + 0.028*access
0.172*change + 0.130*single + 0.079*climate + 0.055*basis + 0.042*problems + 0.040*vary + 0.039*john + 0.037*social + 0.033*displacement + 0.031*resources
0.146*given + 0.110*defined + 0.099*terms + 0.075*amount + 0.074*n + 0.061*position + 0.060*thought + 0.057*x + 0.032*completely + 0.032*respect
0.183*could + 0.173*current + 0.081*nuclei + 0.066*isotopes + 0.050*explain + 0.045*yet +